# install packages
install packages need to train mario agent. Note:
- code will bug with later numpy.

In [ ]:
# !pip install torch
# !pip install matplotlib
!pip install gymnasium==0.29.1
!pip install gym-super-mario-bros==7.4.0
!pip install gym==0.25.2
!pip install imageio-ffmpeg
!pip install imageio
# !pip install torchvision
!pip install opencv-python-headless
!pip install numpy==1.26.4

# import packages

In [ ]:
from PIL import Image
from collections import deque
from datetime import datetime
from pathlib import Path
import copy
import cv2
import imageio
import numpy as np
import random, os
import torch
from torch import nn
import torch.nn.functional as F
import torch.multiprocessing as mp
#import multiprocessing as mp
from torchvision import transforms as T

# Gym is an OpenAI toolkit for RL
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

# NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace

# Super Mario environment for OpenAI Gym
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY

# Create hyperparammeters
Config hyperparammeters, just change it

In [ ]:
#class DictWrapper created by Chatgpt
class DictWrapper:
    def __init__(self, dictionary):
        self._dict = dictionary

    def __getattr__(self, item):
        if item in self._dict:
            return self._dict[item]
        else:
            raise AttributeError(f"'DictWrapper' object has no attribute '{item}'")

config = {
    'num_envs': 32,
    'save_model_step': int(1e5),
    'save_figure_step': 500,
    'learn_step': 20,
    'total_step_or_episode': 'step',
    'total_step': int(5e6),
    'total_episode': None,
    'save_dir': "",
    'gamma': 0.99,
    'learning_rate': 5e-4,
    'state_dim': (4, 84, 84),
    'action_dim': 12, #12 for complex, 7 for simple
    'entropy_coef': 0.01,
    'V_coef': 0.5,
    'world': 1,
    'stage': 1,
    'action_type': 'complex',
    'init_weights': True, #use _initialize_weights function or not
    'kfac_momentum': 0.9,
    'stat_decay': 0.99,
    'kfac_Ts': 1,
    'kfac_Tf': 10,
    'kfac_kl_clip': 0.001,
    'kfac_damping': 1e-2,
    'kfac_fast_cnn': False,
    'weight_decay': 0,
    'gae_lambda': 0.95,
    'eigen_eps': 1e-4 #This parameter can distort the gradient, but it helps the algorithm not to fail when calculating the eigenvalues. Reducing this parameter may make the agent better (no guarantees).
}

config = DictWrapper(config)

In [ ]:
gym.__version__

# Define environment
## Create a custom environment, We need:
- SkipFrame: Because the episode is very long, we only need to repeat actions sometimes in this environment. We repeat each action 4 times and skip the first 3 frames (returning the 4th frame). We also sum the rewards from all 4 frames.
- GrayScaleResizeObservation: Convert the state to grayscale (from RGB to a gray image) and resize it to 84x84 pixels.
- NoopResetEnv: When resetting the environment, we perform random actions before starting the environment. This is similar to the Atari strategy. When resetting, we randomly choose num_noops actions between 0 and noop_max and perform num_noops random actions. If the random actions lead to a terminal state, we reset and continue performing random actions. I set noop_max to 30, similar to Atari.
- CustomRewardAndDoneEnv
    - I noticed that many people train Mario using this custom reward system, so I copied it. The system adds 50 reward points if the agent solves the stage and subtracts 50 reward points if the agent dies. The reward is divided by 10. I set done to True if Mario dies, instead of the default setting where Mario loses all lives.
    - Stage 4-2: Subtract 50 reward points if Mario moves on top of the map (y_pos >= 255).
    - Stages 4-4 and 7-4: Set done = True when Mario goes the wrong way and subtract 50 reward points as a penalty. If Mario takes the correct path but the map still loops (a known bug), I set done = True but do not apply a penalty.
    - Stages 4-4: give a -0.1 reward every step.


## About reward system:
- Set done to True when Mario dies: This is the most important aspect because, in the default reward system, Mario still gains a reward by just moving right. If Mario dies, the agent doesn't lose total rewards and can continue moving right (in the new life) to get more rewards. This is the easiest way for the agent to earn rewards, and it can learn to exploit this trick.
- Penalty of -50 reward when Mario dies: This is necessary to speed up Mario's training. Without this penalty, the agent may struggle to complete more difficult stages.
- Reward of 50 when reaching the flag: This encourages Mario to train faster and overcome difficult sections in harder stages.
- Changing the penalty and flag reward to more or less than 50 doesn't make a significant difference, so I haven't changed it.
- Divide rewards by 10: I believe this reduces the total rewards and helps the agent learn a better strategy, but I'm not entirely sure how necessary this is. I simply followed an existing approach.
- With Stage 4-2: I noticed that the agent can earn more rewards when Mario goes to the warp zone, but Mario can't win this stage using the warp zone because the reward system gives negative rewards when Mario moves left. Therefore, I added a penalty when Mario moves to the top of the map.
- Use FrameStack to stack the latest 4 frames as observation.
- With Stage 4-4 and 7-4:
    - Since this map has a wrong path, Mario can enter a loop where the reward increases indefinitely. To prevent this, I set done = True and assign a negative penalty reward.
    - I also give a negative reward to prevent Mario from taking the wrong path.
    - Another strategy is to give a negative reward without setting done = True (as in 4-2). However, this strategy doesn't work due to a bug in this map.
    - Even when Mario is on the correct path, sometimes he still enters the loop. To handle this, I set done = True every time Mario enters the loop (checked by x_pos and max_x_pos).
- Stage 4-4: Assign a -0.1 reward for every step: This prevents Mario from getting stuck. This map has a section where Mario needs to move left, but moving left incurs a negative reward in the default system. If Mario moves right, he takes the wrong path, causing the episode to end with a negative reward. To keep Mario moving, I added a negative reward for every step.

In [ ]:
class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        for i in range(self._skip):
            # Accumulate reward and repeat the same action
            obs, reward, done, trunk, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, trunk, info

class GrayScaleResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        if isinstance(shape, int):
            self.shape = (shape, shape)
        else:
            self.shape = tuple(shape)

        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        self.current_state = observation
        observation = cv2.cvtColor(observation, cv2.COLOR_RGB2GRAY)
        observation = cv2.resize(observation, self.shape, interpolation=cv2.INTER_AREA)
        observation = observation.astype(np.uint8)#.reshape(-1, observation.shape[0], observation.shape[1])
        return observation

class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        super(NoopResetEnv, self).__init__(env)
        self.noop_max = noop_max

    def reset(self, **kwargs):
        """Do no-op action for a number of steps in [1, noop_max]."""
        obs = self.env.reset(**kwargs)
        noops = np.random.randint(0, self.noop_max, (1, ))[0]
        for _ in range(noops):
            action = self.env.action_space.sample()
            obs, _, done, _, _ = self.env.step(action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        obs, reward, done, trunk, info = self.env.step(ac)
        return obs, reward, done, trunk, info

class CustomRewardAndDoneEnv(gym.Wrapper):
    def __init__(self, env=None, world=1, stage=1, additional_bonus_state_8_4_option = "no"):
        super(CustomRewardAndDoneEnv, self).__init__(env)
        self.current_score = 0
        self.current_x = 0
        self.old_x = -1
        self.current_x_count = 0
        self.max_x = 0
        self.world = world
        self.stage = stage
        if self.world == 8 and self.stage == 4:
            self.sea_map = False
            self.additional_bonus_state_8_4_option = additional_bonus_state_8_4_option

    def reset(self, **kwargs):
        self.current_score = 0
        self.current_x = 0
        self.old_x = -1
        self.current_x_count = 0
        self.max_x = 0
        if self.world == 8 and self.stage == 4:
            self.sea_map = False
        return self.env.reset(**kwargs)

    def step(self, action):
        state, reward, done, trunc, info = self.env.step(action)

        if (info['x_pos'] - self.current_x) == 0:
            self.current_x_count += 1
        else:
            self.current_x_count = 0
        if info["flag_get"]:
            reward += 50
            done = True
        if done and info["flag_get"] == False and info["time"] != 0:
            reward -= 50
            done = True
        self.current_x = info["x_pos"]

        if self.world == 7 and self.stage == 4:
            if (506 <= info["x_pos"] <= 832 and info["y_pos"] > 127) or (
                    832 < info["x_pos"] <= 1064 and info["y_pos"] < 80) or (
                    1113 < info["x_pos"] <= 1464 and info["y_pos"] < 191) or (
                    1579 < info["x_pos"] <= 1943 and info["y_pos"] < 191) or (
                    1946 < info["x_pos"] <= 1964 and info["y_pos"] >= 191) or (
                    1984 < info["x_pos"] <= 2060 and (info["y_pos"] >= 191 or info["y_pos"] < 127)) or (
                    2114 < info["x_pos"] < 2440 and info["y_pos"] < 191):
                reward -= 50
                done = True
            if done == False and info["x_pos"] < self.max_x - 100:
                done = True
        if self.world == 4 and self.stage == 4:
            if (info["x_pos"] <= 1500 and info["y_pos"] < 127) or (
                    1588 <= info["x_pos"] < 2380 and info["y_pos"] >= 127):
                reward -= 50
                done = True
            if done == False and info["x_pos"] < self.max_x - 100:
                done = True
            if done == False:
                reward -= 0.1
        if self.world == 4 and self.stage == 2 and done == False and info['y_pos'] >= 255:
            reward -= 50
        if self.world == 8 and self.stage == 4:
            if info["x_pos"] > 2440 and info["x_pos"] <= 2500:
                done = True
                reward -= 100
            if info["x_pos"] >= 3675 and info["x_pos"] <= 3700:
                done = True
                reward -= 50

            if info["x_pos"] < self.max_x - 200:
                if self.max_x >= 1240 and self.max_x <= 1310: #solved bug because x_pos duplicated
                    if info["x_pos"] >= 320:
                        done = True
                        reward -= 50

            if info["x_pos"] < self.old_x - 200:
                if info["x_pos"] >= 312-5 and info["x_pos"] <= 312+5:
                    done = True
                    reward -= 50
                elif info["x_pos"] >= 56-5 and info["x_pos"] <= 56+5 and self.max_x > 3645 and self.sea_map == False:
                    if self.additional_bonus_state_8_4_option == 'right_pipe':
                        reward += 50
                    self.sea_map = True
            if self.additional_bonus_state_8_4_option == 'right_pipe':
                if info["x_pos"] > self.max_x + 100:
                    reward += 50
            if done == False:
                reward -= 0.1
        self.max_x = max(self.max_x, self.current_x)
        self.current_score = info["score"]
        self.old_x = self.current_x

        return state, reward / 10., done, trunc, info

# Create MultipleEnvironments
MultipleEnvironments use multi-processing to parallel running.

Because in the training process, we need to reset the environment when the agent reaches the terminal state. But if we will do it in parallel, then I don't want to check each environment and reset (by loop) or create a new function that parallels check and reset all environments. Then I reset the environment if done = True in step function and set next_state = env.reset(). Then in training, we just set state = next_state (next_state is reset state if done = True)

In [ ]:
#modify from https://github.com/uvipen/Super-mario-bros-PPO-pytorch/blob/master/src/env.py
def create_env(world, stage, action_type, test=False):
    if gym.__version__ < '0.26':
        env = gym_super_mario_bros.make(f"SuperMarioBros-{world}-{stage}-v0", new_step_api=True)
    else:
        env = gym_super_mario_bros.make(f"SuperMarioBros-{world}-{stage}-v0", render_mode='rgb', apply_api_compatibility=True)

    if action_type == "right":
        action_type = RIGHT_ONLY
    elif action_type == "simple":
        action_type = SIMPLE_MOVEMENT
    else:
        action_type = COMPLEX_MOVEMENT

    env = JoypadSpace(env, action_type)

    if test == False:
        env = NoopResetEnv(env)
    env = SkipFrame(env, skip=4)
    env = CustomRewardAndDoneEnv(env, world, stage)
    env = GrayScaleResizeObservation(env, shape=84)
    if gym.__version__ < '0.26':
        env = FrameStack(env, num_stack=4, new_step_api=True)
    else:
        env = FrameStack(env, num_stack=4)
    return env

class MultipleEnvironments:
    def __init__(self, world, stage, action_type, num_envs):
        self.agent_conns, self.env_conns = zip(*[mp.Pipe(duplex=True) for _ in range(num_envs)])
        self.envs = [create_env(world, stage, action_type) for _ in range(num_envs)]

        for index in range(num_envs):
            process = mp.Process(target=self.run, args=(index,))
            process.start()
            self.env_conns[index].close()

    def run(self, index):
        self.agent_conns[index].close()
        while True:
            request, action = self.env_conns[index].recv()
            if request == "step":
                next_state, reward, done, trunc, info = self.envs[index].step(action)
                if done:
                    next_state = self.envs[index].reset()
                self.env_conns[index].send((next_state, reward, done, trunc, info))
            elif request == "reset":
                self.env_conns[index].send(self.envs[index].reset())
            else:
                raise NotImplementedError

    def step(self, actions):
        [agent_conn.send(("step", act)) for agent_conn, act in zip(self.agent_conns, actions)]
        next_states, rewards, dones, truncs, infos = zip(*[agent_conn.recv() for agent_conn in self.agent_conns])
        return next_states, rewards, dones, truncs, infos

    def reset(self):
        [agent_conn.send(("reset", None)) for agent_conn in self.agent_conns]
        states = [agent_conn.recv() for agent_conn in self.agent_conns]
        return states

In [ ]:
mp.cpu_count()

# Create memory
Memory just save all info we need to train and return all stored info

In [ ]:
class Memory():
    def __init__(self, num_envs):
        self.num_envs = num_envs

        self.states = []
        self.actions = []
        self.next_states = []
        self.rewards = []
        self.dones = []
        self.logits = []
        self.values = []

    def save(self, state, action, reward, next_state, done, logit, V):
        self.states.append(state)
        self.actions.append(action)
        self.next_states.append(next_state)
        self.rewards.append(reward)
        self.dones.append(done)
        self.logits.append(logit)
        self.values.append(V)

    def reset(self):
        self.states = []
        self.actions = []
        self.next_states = []
        self.rewards = []
        self.dones = []
        self.logits = []
        self.values = []

    def get_data(self):
        return self.states, self.actions, self.next_states, self.rewards, self.dones, self.logits, self.values

# KFAC
Edit from https://github.com/ikostrikov/pytorch-a2c-ppo-acktr-gail/blob/master/a2c_ppo_acktr/algo/kfac.py

I changed torch.symeig to torch.linalg.eigh because the new version of torch does not support torch.symeig and torch.symeig is reported to have a bug when calculating gradient.

I added the eigen_eps parameter to make torch.linalg.eigh work. Without eigen_eps, the algorithm will have one of three errors:

- The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated eigenvalues
- This error may appear if the input matrix contains NaN. (nan model)
- Stop without an obvious error (possibly because eigenvalues < 1e-6 leading to no gradient)

In [ ]:
#Edit from https://github.com/ikostrikov/pytorch-a2c-ppo-acktr-gail/blob/master/a2c_ppo_acktr/algo/kfac.py

import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Necessary for my KFAC implementation.
class AddBias(nn.Module):
    def __init__(self, bias):
        super(AddBias, self).__init__()
        self._bias = nn.Parameter(bias.unsqueeze(1))

    def forward(self, x):
        if x.dim() == 2:
            bias = self._bias.t().view(1, -1)
        else:
            bias = self._bias.t().view(1, -1, 1, 1)

        return x + bias

# TODO: In order to make this code faster:
# 1) Implement _extract_patches as a single cuda kernel
# 2) Compute QR decomposition in a separate process
# 3) Actually make a general KFAC optimizer so it fits PyTorch


def _extract_patches(x, kernel_size, stride, padding):
    if padding[0] + padding[1] > 0:
        x = F.pad(x, (padding[1], padding[1], padding[0],
                      padding[0])).data  # Actually check dims
    x = x.unfold(2, kernel_size[0], stride[0])
    x = x.unfold(3, kernel_size[1], stride[1])
    x = x.transpose_(1, 2).transpose_(2, 3).contiguous()
    x = x.view(
        x.size(0), x.size(1), x.size(2),
        x.size(3) * x.size(4) * x.size(5))
    return x


def compute_cov_a(a, classname, layer_info, fast_cnn):
    batch_size = a.size(0)

    if classname == 'Conv2d':
        if fast_cnn:
            a = _extract_patches(a, *layer_info)
            a = a.view(a.size(0), -1, a.size(-1))
            a = a.mean(1)
        else:
            a = _extract_patches(a, *layer_info)
            a = a.view(-1, a.size(-1)).div_(a.size(1)).div_(a.size(2))
    elif classname == 'AddBias':
        is_cuda = a.is_cuda
        a = torch.ones(a.size(0), 1)
        if is_cuda:
            a = a.cuda()

    return a.t() @ (a / batch_size)


def compute_cov_g(g, classname, layer_info, fast_cnn):
    batch_size = g.size(0)

    if classname == 'Conv2d':
        if fast_cnn:
            g = g.view(g.size(0), g.size(1), -1)
            g = g.sum(-1)
        else:
            g = g.transpose(1, 2).transpose(2, 3).contiguous()
            g = g.view(-1, g.size(-1)).mul_(g.size(1)).mul_(g.size(2))
    elif classname == 'AddBias':
        g = g.view(g.size(0), g.size(1), -1)
        g = g.sum(-1)

    g_ = g * batch_size
    return g_.t() @ (g_ / g.size(0))


def update_running_stat(aa, m_aa, momentum):
    # Do the trick to keep aa unchanged and not create any additional tensors
    m_aa *= momentum / (1 - momentum)
    m_aa += aa
    m_aa *= (1 - momentum)


class SplitBias(nn.Module):
    def __init__(self, module):
        super(SplitBias, self).__init__()
        self.module = module
        self.add_bias = AddBias(module.bias.data)
        self.module.bias = None

    def forward(self, input):
        x = self.module(input)
        x = self.add_bias(x)
        return x


class KFACOptimizer(optim.Optimizer):
    def __init__(self,
                 model,
                 lr=0.25,
                 momentum=0.9,
                 stat_decay=0.99,
                 kl_clip=0.001,
                 damping=1e-2,
                 weight_decay=0,
                 fast_cnn=False,
                 Ts=1,
                 Tf=10,
                 eigen_eps=1e-4):
        defaults = dict()

        def split_bias(module):
            for mname, child in module.named_children():
                if hasattr(child, 'bias') and child.bias is not None:
                    module._modules[mname] = SplitBias(child)
                else:
                    split_bias(child)

        split_bias(model)

        super(KFACOptimizer, self).__init__(model.parameters(), defaults)

        self.known_modules = {'Linear', 'Conv2d', 'AddBias'}

        self.modules = []
        self.grad_outputs = {}

        self.model = model
        self._prepare_model()

        self.steps = 0

        self.m_aa, self.m_gg = {}, {}
        self.Q_a, self.Q_g = {}, {}
        self.d_a, self.d_g = {}, {}

        self.momentum = momentum
        self.stat_decay = stat_decay

        self.lr = lr
        self.kl_clip = kl_clip
        self.damping = damping
        self.weight_decay = weight_decay

        self.fast_cnn = fast_cnn

        self.Ts = Ts
        self.Tf = Tf
        self.eigen_eps = eigen_eps

        self.optim = optim.SGD(
            model.parameters(),
            lr=self.lr * (1 - self.momentum),
            momentum=self.momentum)

    def _save_input(self, module, input):
        if torch.is_grad_enabled() and self.steps % self.Ts == 0:
            classname = module.__class__.__name__
            layer_info = None
            if classname == 'Conv2d':
                layer_info = (module.kernel_size, module.stride,
                              module.padding)

            aa = compute_cov_a(input[0].data, classname, layer_info,
                               self.fast_cnn)

            # Initialize buffers
            if self.steps == 0:
                self.m_aa[module] = aa.clone()

            update_running_stat(aa, self.m_aa[module], self.stat_decay)

    def _save_grad_output(self, module, grad_input, grad_output):
        # Accumulate statistics for Fisher matrices
        if self.acc_stats:
            classname = module.__class__.__name__
            layer_info = None
            if classname == 'Conv2d':
                layer_info = (module.kernel_size, module.stride,
                              module.padding)

            gg = compute_cov_g(grad_output[0].data, classname, layer_info,
                               self.fast_cnn)

            # Initialize buffers
            if self.steps == 0:
                self.m_gg[module] = gg.clone()

            update_running_stat(gg, self.m_gg[module], self.stat_decay)

    def _prepare_model(self):
        for module in self.model.modules():
            classname = module.__class__.__name__
            if classname in self.known_modules:
                assert not ((classname in ['Linear', 'Conv2d']) and module.bias is not None), \
                                    "You must have a bias as a separate layer"

                self.modules.append(module)
                module.register_forward_pre_hook(self._save_input)
                module.register_backward_hook(self._save_grad_output)

    def step(self):
        # Add weight decay
        if self.weight_decay > 0:
            for p in self.model.parameters():
                p.grad.data.add_(self.weight_decay, p.data)

        updates = {}
        for i, m in enumerate(self.modules):
            assert len(list(m.parameters())
                       ) == 1, "Can handle only one parameter at the moment"
            classname = m.__class__.__name__
            p = next(m.parameters())

            la = self.damping + self.weight_decay

            if self.steps % self.Tf == 0:
                # My asynchronous implementation exists, I will add it later.
                # Experimenting with different ways to this in PyTorch.
                # self.d_a[m], self.Q_a[m] = torch.symeig(
                #     self.m_aa[m], eigenvectors=True)
                # self.d_g[m], self.Q_g[m] = torch.symeig(
                #     self.m_gg[m], eigenvectors=True)
                self.d_a[m], self.Q_a[m] = torch.linalg.eigh(self.m_aa[m] + self.eigen_eps * torch.eye(self.m_aa[m].size(0), device=self.m_aa[m].device), "U")
                self.d_g[m], self.Q_g[m] = torch.linalg.eigh(self.m_gg[m] + self.eigen_eps * torch.eye(self.m_gg[m].size(0), device=self.m_gg[m].device), "U")

                self.d_a[m].mul_((self.d_a[m] > 1e-6).float())
                self.d_g[m].mul_((self.d_g[m] > 1e-6).float())

            if classname == 'Conv2d':
                p_grad_mat = p.grad.data.view(p.grad.data.size(0), -1)
            else:
                p_grad_mat = p.grad.data

            v1 = self.Q_g[m].t() @ p_grad_mat @ self.Q_a[m]
            v2 = v1 / (
                self.d_g[m].unsqueeze(1) * self.d_a[m].unsqueeze(0) + la)
            v = self.Q_g[m] @ v2 @ self.Q_a[m].t()

            v = v.view(p.grad.data.size())
            updates[p] = v

        vg_sum = 0
        for p in self.model.parameters():
            v = updates[p]
            vg_sum += (v * p.grad.data * self.lr * self.lr).sum()

        nu = min(1, math.sqrt(self.kl_clip / vg_sum))

        for p in self.model.parameters():
            v = updates[p]
            p.grad.data.copy_(v)
            p.grad.data.mul_(nu)

        self.optim.step()
        self.steps += 1

# Create agent
The agent includes 4 main functions:
## train
train function train agent via many episodes:
- loop until the agent wins this stage or reaches the maximum episode/step:
    - predict value, logit for the current state
    - sample action from logit with category distribution (select_action function)
    - log all info to memory
- train agent every learn_step (learn function)
- eval agent every save_figure_step (save_figure function)
- set state = next_state (I reset environment when agent reach terminal state then next_state is first state if done=True)

## select_action
this function sample action from logit:
- just convert logit to probability: policy = F.softmax(logits, dim=1)
- create distribution from probability: distribution = torch.distributions.Categorical(policy)
- sample action from distribution: actions = distribution.sample()

## save_figure
this function eval agent and saves agent/video if the agent yields better total rewards:
- loop until the agent reaches the terminal state.
    - predict logit from model
    - get action = argmax (logit)
    - environment do this action to get next_state, reward, info, done
- if total_reward > best test total reward or agent complete this stage, I save model and video.
- if agent completes this state, we stop training.

## learn
this function trains agent from the experiment saved in memory
- get all the info from memory
- calculate td (lambda) target and gae advantages
- calculate loss:
    - with KFAC: we only need add backward fisher matrix
- update model from loss

In [ ]:
class Agent():
    def __init__(self, world, stage, action_type, envs, num_envs, state_dim, action_dim, save_dir, save_model_step,
                 save_figure_step, learn_step, total_step_or_episode, total_step, total_episode, model,
                 gamma, learning_rate, entropy_coef, V_coef, kfac_momentum, stat_decay, kfac_kl_clip,
                 kfac_damping, weight_decay, kfac_fast_cnn, kfac_Ts, kfac_Tf, gae_lambda, eigen_eps, device):
        self.world = world
        self.stage = stage
        self.action_type = action_type

        self.state_dim = state_dim
        self.action_dim = action_dim
        self.save_dir = save_dir
        self.learn_step = learn_step
        self.total_step_or_episode = total_step_or_episode
        self.total_step = total_step
        self.total_episode = total_episode

        self.current_step = 0
        self.current_episode = 0

        self.save_model_step = save_model_step
        self.save_figure_step = save_figure_step

        self.device = device
        self.save_dir = save_dir

        self.num_envs = num_envs
        self.envs = envs
        self.model = model.to(self.device)

        self.learning_rate = learning_rate
        self.gamma = gamma
        self.gae_lambda = gae_lambda
        self.entropy_coef = entropy_coef
        self.V_coef = V_coef
        self.optimizer = KFACOptimizer(self.model, self.learning_rate, kfac_momentum, stat_decay, kfac_kl_clip,
                                       kfac_damping, weight_decay, kfac_fast_cnn, kfac_Ts, kfac_Tf, eigen_eps)

        self.memory = Memory(self.num_envs)

        self.is_completed = False

        self.env = None
        self.max_test_score = -1e9

        # I just log 1000 lastest update and print it to log.
        self.V_loss = np.zeros((1000,)).reshape(-1)
        self.P_loss = np.zeros((1000,)).reshape(-1)
        self.E_loss = np.zeros((1000,)).reshape(-1)
        self.total_loss = np.zeros((1000,)).reshape(-1)
        self.loss_index = 0
        self.len_loss = 0

    def save_figure(self, is_training=False):
        # test current model and save model/figure if model yield best total rewards.
        # create env for testing, reset test env
        if self.env is None:
            self.env = create_env(self.world, self.stage, self.action_type, True)
        state = self.env.reset()
        done = False

        images = []
        total_reward = 0
        total_step = 0
        num_repeat_action = 0
        old_action = -1

        episode_time = datetime.now()

        # play 1 episode, just get loop action with max probability from model until the episode end.
        while not done:
            with torch.no_grad():
                logit, V = self.model(torch.tensor(state, dtype = torch.float, device = self.device).unsqueeze(0))
            action = logit.argmax(-1).item()
            next_state, reward, done, trunc, info = self.env.step(action)
            state = next_state
            img = Image.fromarray(self.env.current_state)
            images.append(img)
            total_reward += reward
            total_step += 1

            if action == old_action:
                num_repeat_action += 1
            else:
                num_repeat_action = 0
            old_action = action
            if num_repeat_action == 200:
                break

        #logging, if model yield better result, save figure (test_episode.mp4) and model (best_model.pth)
        if is_training:
            f_out = open(f"logging_test.txt", "a")
            f_out.write(f'episode_reward: {total_reward} episode_step: {total_step} current_step: {self.current_step} loss_p: {(self.P_loss.sum()/self.len_loss):.4f} '
                        f'loss_v: {(self.V_loss.sum()/self.len_loss):.4f} loss_e: {(self.E_loss.sum()/self.len_loss):.4f} loss: {(self.total_loss.sum()/self.len_loss):.4f} '
                        f'episode_time: {datetime.now() - episode_time}\n')
            f_out.close()

        if total_reward > self.max_test_score or info['flag_get']:
            imageio.mimsave('test_episode.mp4', images)
            self.max_test_score = total_reward
            if is_training:
                torch.save(self.model.state_dict(), f"best_model.pth")

        # if model can complete this game, stop training by set self.is_completed to True
        if info['flag_get']:
            self.is_completed = True

    def save_model(self):
        torch.save(self.model.state_dict(), f"model_{self.current_step}.pth")

    def load_model(self, model_path = None):
        if model_path is None:
            model_path = f"model_{self.current_step}.pth"
        self.model.load_state_dict(torch.load(model_path))

    def select_action(self, states):
        # select action when training, we need use Categorical distribution to make action base on probability from model
        states = torch.tensor(np.array(states), device = self.device)

        with torch.no_grad():
            logits, V = self.model(states)
            policy = F.softmax(logits, dim=1)
            distribution = torch.distributions.Categorical(policy)
            actions = distribution.sample().cpu().numpy().tolist()
        return actions, logits, V

    def update_loss_statis(self, loss_p, loss_v, loss_e, loss):
        # update loss for logging, just save 1000 latest updates.
        self.V_loss[self.loss_index] = loss_v
        self.P_loss[self.loss_index] = loss_p
        self.E_loss[self.loss_index] = loss_e
        self.total_loss[self.loss_index] = loss
        self.loss_index = (self.loss_index + 1)%1000
        self.len_loss = min(self.len_loss+1, 1000)

    def learn(self):

        # get all data
        states, actions, next_states, rewards, dones, logits, values = self.memory.get_data()

        # calculate target (td lambda target) and gae advantages
        targets = []
        with torch.no_grad():
            _, next_value = self.model(torch.tensor(np.array(next_states[-1]), device = self.device))
        target = next_value
        advantage = 0

        for state, next_state, reward, done, V in zip(states[::-1], next_states[::-1], rewards[::-1], dones[::-1], values[::-1]):
            done = torch.tensor(done, device = self.device, dtype = torch.float).reshape(-1, 1)
            reward = torch.tensor(reward, device = self.device).reshape(-1, 1)

            target = next_value * self.gamma * (1-done) + reward
            advantage = target + self.gamma * advantage * (1-done) * self.gae_lambda
            targets.append(advantage)
            advantage = advantage - V.detach()
            next_value = V.detach()
        targets = targets[::-1]

        states = torch.tensor(np.array(states), device = self.device)
        states = states.reshape(-1, states.shape[2], states.shape[3], states.shape[4])

        logits, values = self.model(states)
        index = torch.arange(0, len(logits), device = self.device)
        actions = torch.flatten(torch.tensor(actions, device = self.device, dtype = torch.int64))

        targets = torch.cat(targets, 0).view(-1, 1)
        dist = torch.distributions.Categorical(logits = logits)
        log_probs = dist.log_prob(actions)
        entropy = dist.entropy().mean()

        advantages = (targets - values).reshape(-1)
        loss_V = F.mse_loss(values, targets)

        loss_P = -(log_probs * advantages.detach()).mean()

        # backward fisher matrix (This is the only addition over A2C's Adam)
        if self.optimizer.steps % self.optimizer.Ts == 0:
            self.model.zero_grad()
            pg_fisher_loss = -log_probs.mean()  # CELoss
            value_noise = torch.randn(values.size()).to(self.device)
            sample_values = values + value_noise
            vf_fisher_loss = - (values - sample_values.detach()).pow(2).mean()
            fisher_loss = pg_fisher_loss + vf_fisher_loss
            self.optimizer.acc_stats = True
            fisher_loss.backward(retain_graph=True)
            self.optimizer.acc_stats = False

        self.optimizer.zero_grad()
        loss = - entropy * self.entropy_coef + loss_V * self.V_coef + loss_P
        loss.backward()

        self.optimizer.step()

        self.update_loss_statis(loss_P.item(), loss_V.item(), entropy.item(), loss.item())

    def train(self):
        episode_reward = [0] * self.num_envs
        episode_step = [0] * self.num_envs
        max_episode_reward = 0
        max_episode_step = 0
        episode_time = [datetime.now() for _ in range(self.num_envs)]
        total_time = datetime.now()

        last_episode_rewards = []

        # reset envs
        states = self.envs.reset()

        while True:
            # finish training if agent reach total_step or total_episode based on what type of total_step_or_episode is step or episode
            self.current_step += 1

            if self.total_step_or_episode == 'step':
                if self.current_step >= self.total_step:
                    break
            else:
                if self.current_episode >= self.total_episode:
                    break

            actions, logit, V = self.select_action(states)

            next_states, rewards, dones, truncs, infos = self.envs.step(actions)

            # save to memory
            self.memory.save(states, actions, rewards, next_states, dones, logit, V)

            episode_reward = [x + reward for x, reward in zip(episode_reward, rewards)]
            episode_step = [x+1 for x in episode_step]

            # logging after each step, if 1 episode is ending, I will log this to logging.txt
            for i, done in enumerate(dones):
                if done:
                    self.current_episode += 1
                    max_episode_reward = max(max_episode_reward, episode_reward[i])
                    max_episode_step = max(max_episode_step, episode_step[i])
                    last_episode_rewards.append(episode_reward[i])
                    f_out = open(f"logging.txt", "a")
                    f_out.write(f'episode: {self.current_episode} agent: {i} rewards: {episode_reward[i]:.4f} steps: {episode_step[i]} complete: {infos[i]["flag_get"]==True} '
                                f'mean_rewards: {np.array(last_episode_rewards[-min(len(last_episode_rewards), 100):]).mean():.4f} max_rewards: {max_episode_reward:.4f} '
                                f'max_steps: {max_episode_step} current_step: {self.current_step} loss_p: {(self.P_loss.sum()/self.len_loss):.4f} '
                                f'loss_v: {(self.V_loss.sum()/self.len_loss):.4f} loss_e: {(self.E_loss.sum()/self.len_loss):.4f} loss: {(self.total_loss.sum()/self.len_loss):.4f} '
                                f'episode_time: {datetime.now() - episode_time[i]} total_time: {datetime.now() - total_time}\n')
                    f_out.close()
                    episode_reward[i] = 0
                    episode_step[i] = 0
                    episode_time[i] = datetime.now()

            # training agent every learn_step
            if self.current_step % self.learn_step == 0:
                self.learn()
                self.memory.reset()

            if self.current_step % self.save_model_step == 0:
                self.save_model()

            # eval agent every save_figure_step
            if self.current_step % self.save_figure_step == 0 and self.save_figure_step != -1:
                self.save_figure(is_training=True)
                if self.is_completed:
                    f_out = open(f"logging.txt", "a")
                    f_out.write(f' mean_rewards: {np.array(last_episode_rewards[-min(len(last_episode_rewards), 100):]).mean()} max_rewards: {max_episode_reward} '
                                f'max_steps: {max_episode_step} current_step: {self.current_step} total_time: {datetime.now() - total_time}\n')
                    f_out.close()
                    return

            states = list(next_states)

        f_out = open(f"logging.txt", "a")
        f_out.write(f' mean_rewards: {np.array(last_episode_rewards[-min(len(last_episode_rewards), 100):]).mean()} max_rewards: {max_episode_reward} '
                    f'max_steps: {max_episode_step} current_step: {self.current_step} total_time: {datetime.now() - total_time}\n')
        f_out.close()

# Create model
Model includes:
- 4 convolution layers to encode input image (observation) to feature vector.
- 1 linear layers (because KFAC is not implemented for RNN or it is very difficult and does not guarantee that the available KFAC RNN code is bug-free)
- two linear layers for policy and value prediction (actor and critic)

In [ ]:
class Model(nn.Module):
    def __init__(self, input_dim, output_dim, init_weights):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(input_dim[0], 32, 3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.linear = nn.Linear(1152, 512)
        self.critic_linear = nn.Linear(512, 1)
        self.actor_linear = nn.Linear(512, output_dim)
        if init_weights:
            self._initialize_weights()

    def _initialize_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                nn.init.orthogonal_(module.weight)
                # nn.init.kaiming_uniform_(module.weight)
                nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.LSTMCell):
                nn.init.constant_(module.bias_ih, 0)
                nn.init.constant_(module.bias_hh, 0)

    def forward(self, x):
        x = F.relu(self.conv1(x/255.))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.linear(x))
        return self.actor_linear(x), self.critic_linear(x)

# train

In [ ]:
model = Model(config.state_dim, config.action_dim, config.init_weights)

In [ ]:
envs = MultipleEnvironments(config.world, config.stage, config.action_type, config.num_envs)

In [ ]:
envs.envs[0]

In [ ]:
agent = Agent(world = config.world, stage = config.stage, action_type = config.action_type, envs = envs, num_envs = config.num_envs,
                  state_dim = config.state_dim, action_dim = config.action_dim, save_dir = config.save_dir,
                  save_model_step = config.save_model_step, save_figure_step = config.save_figure_step, learn_step = config.learn_step,
                  total_step_or_episode = config.total_step_or_episode, total_step = config.total_step, total_episode = config.total_episode,
                  model = model, gamma = config.gamma, learning_rate = config.learning_rate, entropy_coef = config.entropy_coef, 
                  V_coef = config.V_coef, kfac_momentum = config.kfac_momentum, stat_decay = config.stat_decay, kfac_kl_clip = config.kfac_kl_clip, 
                  kfac_damping = config.kfac_damping, weight_decay = config.weight_decay, kfac_fast_cnn = config.kfac_fast_cnn, 
                  kfac_Ts = config.kfac_Ts, kfac_Tf = config.kfac_Tf, gae_lambda = config.gae_lambda,
                  eigen_eps = config.eigen_eps, device = "cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
agent.train()

In [ ]:
agent.current_step

# test

In [ ]:
agent.load_model("best_model.pth")
agent.save_figure()